# Install the required packages

In [1]:
! pip install langgraph openai python-dotenv  --quiet

# Generate python code

In [2]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import json


load_dotenv()
# Ensure you have your OpenAI API key set

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

class State:
    def __init__(self):
        self.data = {}
        self.attempts = 0  # To keep track of code generation attempts

    def update(self, new_data):
        self.data.update(new_data)

    def get(self, key):
        return self.data.get(key)

    def increment_attempts(self):
        self.attempts += 1

    def get_attempts(self):
        return self.attempts

class Node:
    def __init__(self, name):
        self.name = name

    def run(self, state):
        raise NotImplementedError

class AgentNode(Node):
    def __init__(self):
        super().__init__("agent")

    def run(self, state):
        state.update({"instructions": "Write a Python script that prints out \"Hello World\"."})
        return state

class CodeGeneratorNode(Node):
    MAX_ATTEMPTS = 1

    def __init__(self):
        super().__init__("code_generator")

    def run(self, state):
        if state.get_attempts() >= self.MAX_ATTEMPTS:
            print("Max code generation attempts reached, failing the node.")
            state.update({"failed": True})
            return state

        instructions = state.get("instructions")
        if instructions:
            state.increment_attempts()
            # Generate code using OpenAI GPT-4 Turbo model with function calling
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": instructions}
                ],
                functions=[
                    {
                        "name": "generated_python_code",
                        "description": "Python code",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "python_code": {"type": "string"}
                            },
                            "required": ["python_code"],
                        }
                    }
                ],
            )
            print("Response:", response.choices[0].message)
            argument = response.choices[0].message.function_call.arguments
            data = json.loads(argument)
            code = data["python_code"]
            print("Generated code:", code)
            state.update({"python_code": code})
        return state

class SaveCodeNode(Node):
    def __init__(self):
        super().__init__("save_code")

    def run(self, state):
        python_code = state.get("python_code")
        if python_code:
            try:
                directory = "generated"
                file_name = "generated_script.py"
                if not os.path.exists(directory):
                    os.makedirs(directory)
                with open(os.path.join(directory, file_name), "w") as file:
                    file.write(python_code)
                state.update({"file_saved": True})
            except Exception as e:
                print(f"Saving code failed with error: {e}")
                state.update({"retry": True})
        return state

class ExecuteCodeNode(Node):
    def __init__(self):
        super().__init__("execute_code")

    def run(self, state):
        try:
            exec(state.get("python_code"))
            state.update({"success": True})
        except Exception as e:
            print(f"Execution failed with error: {e}")
            state.update({"retry": True})
        return state

class StateGraph:
    def __init__(self):
        self.nodes = {}
        self.entry_point = None
        self.edges = {}

    def add_node(self, name, node):
        self.nodes[name] = node

    def set_entry_point(self, name):
        self.entry_point = name

    def add_edge(self, from_node, to_node):
        if from_node not in self.edges:
            self.edges[from_node] = []
        self.edges[from_node].append(to_node)

    def compile(self):
        return self

    def run(self, state):
        current_node = self.entry_point
        while True:
            node = self.nodes[current_node]
            state = node.run(state)
            if state.get("success"):
                return state
            elif state.get("failed"):
                print("Node failed, ending the graph as failed.")
                return state
            elif state.get("retry"):
                current_node = "code_generator"
            else:
                next_nodes = self.edges.get(current_node)
                if not next_nodes:
                    break
                current_node = next_nodes[0]

# Initialize the graph
workflow = StateGraph()

# Add the nodes to the graph
workflow.add_node("agent", AgentNode())
workflow.add_node("code_generator", CodeGeneratorNode())
workflow.add_node("save_code", SaveCodeNode())
workflow.add_node("execute_code", ExecuteCodeNode())

# Set the entry point to the "agent" node
workflow.set_entry_point("agent")

# Add edges to form the workflow
workflow.add_edge("agent", "code_generator")
workflow.add_edge("code_generator", "save_code")
workflow.add_edge("save_code", "execute_code")
workflow.add_edge("execute_code", "code_generator")

# Compile the graph into a runnable
app = workflow.compile()

# Example usage
state = State()
state = app.run(state)
if state.get("success"):
    print("Execution successful, ending the graph.")
elif state.get("failed"):
    print("Execution failed after maximum attempts.")
else:
    print("Unexpected output:", state.data)


Response: ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "python_code": "print(\'Hello World\')"\n}', name='generated_python_code'), tool_calls=None)
Generated code: print('Hello World')
Hello World
Execution successful, ending the graph.
